### Example of multioutput model using the functional API - Housing dataset

#### Loading the dataset and standardizing data

In [1]:
# Boston Housing dataset - load data
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

# standardize the data
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

(404, 13)
(102, 13)
50.0 5.0


#### Create, train and estimate the error of a model for the Housing dataset using the functional API

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# create model using functional API
input_tensor = Input(shape=(train_data.shape[1],))
x = layers.Dense(64, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
output_tensor = layers.Dense(1,activation= "linear")(x)

model = Model(input_tensor, output_tensor)
model.summary()

# train and test
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.fit(train_data, train_targets, epochs=20, batch_size=16, verbose=1)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
26/26 [==============================] - 0s 769us/step - loss: 491.6964 - mae: 20.1852
Epoch 2/20
26/26 [==============================] - 0s 774us/step - loss: 317.0489 - mae: 15.6194
Epoch 3/20
26/26 [==============================] - 0s

Now we create a binary variable to transform the model to a multi-output problem:

In [3]:
import numpy as np

# In computational geometry, the bin is a data structure that allows efficient region queries. Each time a data point falls into a bin, the frequency of that bin is increased by one.
# create binary variable by returning the indexes of the bin to which the value belongs.
# where the bin is defined as 20 with a right as False, meaning it returns the price higher than 20
train_targets_bin = np.digitize(train_targets,bins=[20])
test_targets_bin = np.digitize(test_targets,bins=[20])

In [4]:
# model with two different outputs: regression and binary classification

input_tensor = Input(shape=(train_data.shape[1],))
x = layers.Dense(64, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
output_tensor1 = layers.Dense(1,activation= "linear", name= "output1")(x)
output_tensor2 = layers.Dense(1,activation= "sigmoid", name = "output2")(x)

model = Model(input_tensor, [output_tensor1, output_tensor2])
model.summary()

model.compile(optimizer='rmsprop', loss=['mse','binary_crossentropy'], loss_weights = [0.05, 1])
# define the weights for the loss function of each exit

model.fit(train_data, [train_targets, train_targets_bin], epochs=20, batch_size=16)

print(model.evaluate(test_data, [test_targets, test_targets_bin]) )

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 64)           896         input_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 64)           4160        dense_3[0][0]                    
__________________________________________________________________________________________________
output1 (Dense)                 (None, 1)            65          dense_4[0][0]                    
_______________________________________________________________________________________